# ORMs

#### What is an ORM?
> [Object Relational Mapping](https://en.wikipedia.org/wiki/Object-relational_mapping)(ORM) is a technique that lets you query and manipulate data from a database using an object-oriented paradigm. When talking about ORM, most people are referring to a library that implements the Object-Relational Mapping technique, hence the phrase "an ORM".

> An ORM library a completely ordinary library written in your language of choice that encapsulates the code needed to manipulate the data, so you don't use SQL anymore; you interact directly with an object in the same language you're using.

### Pros
##### Pro # 1: It saves a lot of time
> - [DRY](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself): You write your data model in only one place, and it's easier to update, maintain, and reuse the code.

> - A lot of stuff is done automatically, from database handling to I18N.
- It forces you to write MVC code, which, in the end, makes your code a little cleaner.
- You don't have to write poorly-formed SQL (most Web programmers really suck at it, because SQL is treated like a "sub" language, when in reality it's a very powerful and complex one)
- Sanitizing; using prepared statements or transactions are as easy as calling a method.

##### Pro # 2: It is more flexible
> - It fits in your natural way of coding (it's your language!)
- It abstracts the DB system, so you can change it whenever you want.
- The model is weakly bound to the rest of the application, so you can change it or use it anywhere else.
- It lets you use OOP goodness like data inheritance without a headache.


### Con
##### It can be a pain
> - You have to learn it, and ORM libraries are not lightweight tools;
- You have to set it up. Same problem.
- Performance is OK for usual queries, but a SQL master will always do better with his own SQL for big projects.
- It abstracts the DB. While it's OK if you know what's happening behind the scene, it's a trap for new programmers that can write very greedy statements, like a heavy hit in a <b>`for`</b> loop.

## 18.1 PeeWee

## 18.2 SQLAlchemy

> SQLAlchemy is the Python SQL toolkit and Object Relational Mapper that gives application developers the full power and flexibility of SQL.

> SQL databases behave less like object collections the more size and performance start to matter; object collections behave less like tables and rows the more abstraction starts to matter. SQLAlchemy aims to accommodate both of these principles.

>SQLAlchemy considers the database to be a relational algebra engine, not just a collection of tables. Rows can be selected from not only tables but also joins and other select statements; any of these units can be composed into a larger structure. SQLAlchemy's expression language builds on this concept from its core.

> The main goal of SQLAlchemy is to change the way you think about databases and SQL!

##### Version Check
We should be on at least <b>version 1.1</b> of SQLAlchemy:

In [ ]:
import sqlalchemy
sqlalchemy.__version__ 

##### Connecting
For this tutorial we will use an in-memory-only SQLite database. To connect we use <b>`create_engine()`</b>:

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

<b>`echo`</b> 
- This is shortcut to setting up SQLAlchemy logging, which is accomplished via Python’s standard <b>`logging`</b> module. 
- With it enabled, we’ll see all the generated SQL produced. <i>(If you are working through this tutorial and want less output generated, set it to <b>`False`</b>. )</i>

<b>`create_engine()`</b>
- The return value of <b>`create_engine()`</b> is an instance of <b>`Engine`</b> and it represents the core interface to the database.

> The `Engine`, when first returned by `create_engine()`, has not actually tried to connect to the database yet; that happens only the first time it is asked to perform a task against the database.

##### Declare a Mapping

- When using the ORM, we need to describe the database tables we'll be dealing with, and then by defining our own classes, which will be mapped to those tables.

- In modern SQLAlchemy, these two tasks are usually performed together, using a system known as `Declarative`, which allows us to create classes that include directives to describe the actual database table they will be mapped to.

In [ ]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

Now that we have a “base”, we can define any number of mapped classes in terms of it. We will start with just a single table called users, which will store records for the end-users using our application. A new class called User will be the class to which we map this table. Within the class, we define details about the table to which we’ll be mapping, primarily the table name, and names and datatypes of columns:

In [ ]:
from sqlalchemy import Column, Integer, String
class User(Base):
        __tablename__ = 'users'

        id = Column(Integer, primary_key=True)
        name = Column(String)
        fullname = Column(String)
        password = Column(String)

        def __repr__(self):
            return "<User(name='%s', fullname='%s', password='%s')>" % (
                self.name, self.fullname, self.password)

- A class using Declarative at a minimum needs a <b>`__tablename__`</b> attribute, and at least one Column which is part of a primary key.

- SQLAlchemy never makes any assumptions by itself about the table to which a class refers, including that it has no built-in conventions for names, datatypes, or constraints.

- But this doesn’t mean boilerplate is required; instead, you’re encouraged to create your own automated conventions using helper functions and mixin classes, which is described in detail at [Mixin and Custom Base Classes](http://docs.sqlalchemy.org/en/latest/orm/extensions/declarative/mixins.html#declarative-mixins).

- When our class is constructed, Declarative replaces all the Column objects with special Python accessors known as descriptors; this is a process known as instrumentation. The “instrumented” mapped class will provide us with the means to refer to our table in a SQL context as well as to persist and load the values of columns from the database.

- Outside of what the mapping process does to our class, the class remains otherwise mostly a normal Python class, to which we can define any number of ordinary attributes and methods needed by our application.

##### Declare a Schema

With our User class constructed via the Declarative system, we have defined information about our table, known as table metadata. The object used by SQLAlchemy to represent this information for a specific table is called the Table object, and here Declarative has made one for us. We can see this object by inspecting the <b>`__table__`</b> attribute:

In [ ]:
User.__table__

![Intermediate_3](./images/img_user_table.png)

- When we declared our class, Declarative used a Python metaclass in order to perform additional activities once the class declaration was complete; within this phase, it then created a <b>`Table`</b> object according to our specifications, and associated it with the class by constructing a <b>`Mapper`</b> object.


- This object is a behind-the-scenes object we normally don’t need to deal with directly (though it can provide plenty of information about our mapping when we need it).

- The Table object is a member of a larger collection known as <b>`MetaData`</b>. When using Declarative, this object is available using the <b>`.metadata`</b> attribute of our declarative base class.


- The MetaData is a registry which includes the ability to emit a limited set of schema generation commands to the database. As our SQLite database does not actually have a users table present, we can use MetaData to issue <b>`CREATE TABLE`</b> statements to the database for all tables that don’t yet exist. Below, we call the MetaData.create_all() method, passing in our Engine as a source of database connectivity. We will see that special commands are first emitted to check for the presence of the users table, and following that the actual CREATE TABLE statement:

In [ ]:
Base.metadata.create_all(engine)

![Intermediate_3](./images/img_meta_data.png)

##### Create an Instance of the Mapped Class

With mappings complete, let’s now create and inspect a <b>`User`</b> object:

In [ ]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')
ed_user.name

![Intermediate_3](./images/img_ed.png)

In [ ]:
ed_user.password

str(ed_user.id)

![Intermediate_3](./images/img_ed_pw.png)

- Even though we didn’t specify it in the constructor, the id attribute still produces a value of None when we access it (as opposed to Python’s usual behavior of raising AttributeError for an undefined attribute). 


- SQLAlchemy’s instrumentation normally produces this default value for column-mapped attributes when first accessed. For those attributes where we’ve actually assigned a value, the instrumentation system is tracking those assignments for use within an eventual INSERT statement to be emitted to the database.



##### Creating a Session

We’re now ready to start talking to the database. The ORM’s “handle” to the database is the <b>`Session`</b>. When we first set up the application, at the same level as our <b>`create_engine()`</b> statement, we define a Session class which will serve as a factory for new <b>`Session`</b> objects:

In [ ]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

In the case where your application does not yet have an Engine when you define your module-level objects, just set it up like this:

In [ ]:
Session = sessionmaker()

Later, when you create your engine with create_engine(), connect it to the Session using configure():

In [ ]:
Session.configure(bind=engine)  # once engine is available

This custom-made <b>`Session`</b> class will create new <b>`Session`</b> objects which are bound to our database. Other transactional characteristics may be defined when calling <b>`sessionmaker`</b> as well; these are described in a later chapter. Then, whenever you need to have a conversation with the database, you instantiate a <b>`Session`</b>:

In [ ]:
session = Session()

The above Session is associated with our SQLite-enabled Engine, but it hasn’t opened any connections yet. When it’s first used, it retrieves a connection from a pool of connections maintained by the Engine, and holds onto it until we commit all changes and/or close the session object.

##### Adding and Updating objects

To persist our <b>`User`</b> object, we add() it to our <b>`Session`</b>:

In [ ]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')
session.add(ed_user)

At this point, we say that the instance is pending; no SQL has yet been issued and the object is not yet represented by a row in the database. The Session will issue the SQL to persist Ed Jones as soon as is needed, using a process known as a flush. If we query the database for Ed Jones, all pending information will first be flushed, and the query is issued immediately thereafter.

For example, below we create a new Query object which loads instances of User. We “filter by” the name attribute of ed, and indicate that we’d like only the first result in the full list of rows. A User instance is returned which is equivalent to that which we’ve added:

In [ ]:
our_user = session.query(User).filter_by(name='ed').first() 

![Intermediate_3](./images/img_our_user.png)

In [ ]:
our_user

![Intermediate_3](./images/img_our_user_ed.png)

In fact, the Session has identified that the row returned is the same row as one already represented within its internal map of objects, so we actually got back the identical instance as that which we just added:

![Intermediate_3](./images/img_ed_our_user_true.png)

The ORM concept at work here is known as an identity map and ensures that all operations upon a particular row within a Session operate upon the same set of data. Once an object with a particular primary key is present in the Session, all SQL queries on that Session will always return the same Python object for that particular primary key; it also will raise an error if an attempt is made to place a second, already-persisted object with the same primary key within the session.

We can add more User objects at once using add_all():

In [ ]:
session.add_all([
    User(name='wendy', fullname='Wendy Williams', password='foobar'),
    User(name='mary', fullname='Mary Contrary', password='xxg527'),
    User(name='fred', fullname='Fred Flinstone', password='blah')])

Also, we’ve decided the password for Ed isn’t too secure, so lets change it:

In [ ]:
ed_user.password = 'f8s7ccs'

The Session is paying attention. It knows, for example, that Ed Jones has been modified:

In [ ]:
session.dirty

![Intermediate_3](./images/img_session_dirty.png)

and that three new User objects are pending:

In [ ]:
session.new

![Intermediate_3](./images/img_session_new.png)

We tell the Session that we’d like to issue all remaining changes to the database and commit the transaction, which has been in progress throughout. We do this via commit(). The Session emits the UPDATE statement for the password change on “ed”, as well as INSERT statements for the three new User objects we’ve added:

In [ ]:
session.commit()

![Intermediate_3](./images/img_fake_user.png)

commit() flushes whatever remaining changes remain to the database, and commits the transaction. The connection resources referenced by the session are now returned to the connection pool. Subsequent operations with this session will occur in a new transaction, which will again re-acquire connection resources when first needed.

If we look at Ed’s id attribute, which earlier was None, it now has a value:

In [ ]:
ed_user.id 

![Intermediate_3](./images/img_ed_user_1.png)

After the Session inserts new rows in the database, all newly generated identifiers and database-generated defaults become available on the instance, either immediately or via load-on-first-access. In this case, the entire row was re-loaded on access because a new transaction was begun after we issued commit(). SQLAlchemy by default refreshes data from a previous transaction the first time it’s accessed within a new transaction, so that the most recent state is available.

##### Rolling Back

Since the Session works within a transaction, we can roll back changes made too. Let’s make two changes that we’ll revert; ed_user‘s user name gets set to Edwardo:

In [ ]:
ed_user.name = 'Edwardo'

and we’ll add another erroneous user, fake_user:

In [ ]:
fake_user = User(name='fakeuser', fullname='Invalid', password='12345')
session.add(fake_user)

Querying the session, we can see that they’re flushed into the current transaction:

In [ ]:
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

Rolling back, we can see that ed_user‘s name is back to ed, and fake_user has been kicked out of the session:

In [ ]:
session.rollback()

![Intermediate_3](./images/img_rollback.png)

In [ ]:
ed_user.name

![Intermediate_3](./images/img_ed_username.png)

In [ ]:
fake_user in session

![Intermediate_3](./images/img_fake_user_false.png)

issuing a SELECT illustrates the changes made to the database:

In [ ]:
session.query(User).filter(User.name.in_(['ed', 'fakeuser'])).all()

![Intermediate_3](./images/img_query_user.png)

##### Querying

- A Query object is created using the <b>`query()`</b> method on <b>`Session`</b>. 

- This function takes a variable number of arguments, which can be any combination of classes and class-instrumented descriptors. Below, we indicate a Query which loads User instances. When evaluated in an iterative context, the list of User objects present is returned:

In [ ]:
for instance in session.query(User).order_by(User.id):
        print(instance.name, instance.fullname)

![Intermediate_3](./images/img_return_user_object.png)

You can control the names of individual column expressions using the label() construct, which is available from any ColumnElement-derived object, as well as any class attribute which is mapped to one (such as User.name):

In [ ]:
for row in session.query(User.name.label('name_label')).all():
    print(row.name_label)

![Intermediate_3](./images/img_row_label.png)

The name given to a full entity such as User, assuming that multiple entities are present in the call to query(), can be controlled using aliased() :

In [ ]:
>>> from sqlalchemy.orm import aliased
>>> user_alias = aliased(User, name='user_alias')

>>> for row in session.query(user_alias, user_alias.name).all():
    print(row.user_alias)

![Intermediate_3](./images/img_alias.png)

Basic operations with Query include issuing LIMIT and OFFSET, most conveniently using Python array slices and typically in conjunction with ORDER BY:

In [ ]:
for u in session.query(User).order_by(User.id)[1:3]:
    print(u)

![Intermediate_3](./images/img_offset_limit.png)

and filtering results, which is accomplished either with filter_by(), which uses keyword arguments:

In [ ]:
for name, in session.query(User.name).\
        filter_by(fullname='Ed Jones'):
    print(name)

![Intermediate_3](./images/img_filter_by_ed.png)

or filter(), which uses more flexible SQL expression language constructs. These allow you to use regular Python operators with the class-level attributes on your mapped class:

In [ ]:
for name, in session.query(User.name).\
            filter(User.fullname=='Ed Jones'):
    print(name)

![Intermediate_3](./images/img_filter_ed.png)